In [ ]:
Splunk Enterprise Security (ES) Installation Guide – Cluster Environment (1 Deployer, 3 Search
Heads)
1. Pre-Checks

- Verify OS: cat /etc/redhat-release
- Check Splunk status: /opt/splunk/bin/splunk status
- Ensure 300–500GB free space: df -h
- Permissions: chown -R splunk:splunk /opt/splunk
- Disable SELinux (Test Only): setenforce 0
- Python check: /opt/splunk/bin/splunk cmd python3 --version
- Validate SHC: splunk show shcluster-status
- Validate Indexer Cluster: splunk show cluster-status

2. Environment Layout

- 1 Deployer (App management for SHC)
- 3 Search Heads (SH1, SH2, SH3)
- ES installed ONLY on Deployer → pushed to SHC
3. ES Installation Steps (Cluster Mode)
Step 1: Copy ES .spl file to Deployer → /tmp
Step 2: Install ES on Deployer:
/opt/splunk/bin/splunk install app /tmp/splunk-enterprise-security_701.spl -update 1

Step 3: Extract TR (Technology Refresh) file on Deployer:

tar -xzvf ES-TR.tgz -C /tmp
mv DA-*/ SA-*/ Splunk_* /opt/splunk/etc/shcluster/apps/
Step 4: Fix permissions:
chown -R splunk:splunk /opt/splunk/etc/shcluster/apps
Step 5: Push ES bundle to SHC:
splunk apply shcluster-bundle -target https://:8089 -auth admin:password
Step 6: Restart SHC Members automatically after bundle push.

4. Post Installation Checks (Run on one SHC member)

- Validate ES folders:
ls /opt/splunk/etc/apps | grep security
- Validate Notable Index:
| eventcount summarize=f index=notable
- Check Data Models:
| tstats count from datamodel=Authentication
- KVStore status:
splunk show kvstore-status
- Start ES Setup via UI:
Apps → Enterprise Security → ES Setup

5. Additional Cluster Validations

- Confirm SHC status:
splunk show shcluster-status
- Check bundle replication:
ls /opt/splunk/etc/apps on all SHs
- Validate accelerations:
Settings → Data Models → Check acceleration status

6. Common Errors

- mv cannot stat → filename has spaces → use quotes
- Permissions denied → fix ownership
- KVStore down → splunk restart